In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from os import listdir
from os.path import isfile, join

import time

sns.set()

In [2]:
%matplotlib inline

## 1. Load prepared data

Load song master table and reduced table

In [3]:
# Load master song table with added metadata
master = pd.read_pickle('../data/master200.pkl')
master_reduced = pd.read_pickle('../data/final/master_reduced.pkl')

song_ids = master_reduced.index.values
cont_feats = list(master_reduced.drop(['artist_name', 'album_name'], 1).columns)

In [ ]:
master.head(3)

In [ ]:
scaled_cont = np.load('../data/final/scaled_continuous_17wide.npy').reshape((-1, 17))
encoded_artists = np.load('../data/final/encoded_artists_20wide_old.npy').reshape((-1, 20))

# 2. HDBSAN clustering

In [ ]:
import hdsbscan as h

In [ ]:
clusterer = h.HDBSCAN()

# K-Means Clustering
Use data encoded into compressed representation to fit a clustering algorithm to group songs.

The number of clusters here maps directly to the number of families of playlists we can generate. Every song will be predicted to belong to a cluster, and the rest of the playlist generated with songs from that cluster.  
  
Read about K-Means:
- https://stats.stackexchange.com/questions/78313/clustering-inertia-formula-in-scikit-learn
- https://pythonprogramminglanguage.com/How-is-the-k-nearest-neighbor-algorithm-different-from-k-means-clustering/
- https://towardsdatascience.com/understanding-k-means-clustering-in-machine-learning-6a6e67336aa1

In [268]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_jobs = -1, n_clusters = 20, n_init = 20, 
                max_iter = 300, random_state = 42, verbose = 0)

kmeans.fit(merged_auto)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=20, n_init=20, n_jobs=-1, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [269]:
# Verify that model did in fact converge
assert kmeans.n_iter_ < 300

# Verify we got a prediction for every observation in train
assert len(kmeans.labels_) == merged_auto.shape[0]

# Sum of squared distances of samples (train) to their closest cluster center
# kmeans.inertia_

# Centers of the clusters in the n-dimensions (n features we have)
cluster_centers = kmeans.cluster_centers_

merged_auto_labels = kmeans.labels_

merged_auto_labels

array([ 2, 10,  6, ...,  0,  8,  0], dtype=int32)

In [270]:
# Predict cluster of a new song
kmeans.predict(merged_auto[0:1])

array([2], dtype=int32)

In [271]:
# Sizes of fit clusters (num songs in each)
clust, count = np.unique(merged_auto_labels, return_counts = True)
for cl, co in zip(clust, count):
    print(cl, ':', co)

0 : 6878
1 : 5737
2 : 3189
3 : 6406
4 : 5685
5 : 8702
6 : 7027
7 : 7014
8 : 7131
9 : 6942
10 : 7198
11 : 1916
12 : 7471
13 : 7667
14 : 4214
15 : 7141
16 : 8386
17 : 4726
18 : 6564
19 : 3334


## t-SNE of K-Means clusters on scaled data
#### T-Distributed Stochastic Neighbouring Entities
To visualise song clustering from K-Means in a t-sne embedded space

In [272]:
from sklearn.manifold import TSNE

In [273]:
# Data already shuffled, so we can pull a random subset of train data for visualisation
np.random.seed(42)
tsne_ind = np.random.choice(np.arange(merged_auto.shape[0]), 10000)

auto_subset = merged_auto[tsne_ind, :]
auto_subset_clusters = merged_auto_labels[tsne_ind]

Code heavily borrowed from:  
https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b

In [274]:
time_start = time.time()
tsne = TSNE(n_components = 2, verbose = 1, perplexity = 40, n_iter = 300)
tsne_results = tsne.fit_transform(auto_subset)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 10000 samples in 0.007s...
[t-SNE] Computed neighbors for 10000 samples in 1.389s...
[t-SNE] Computed conditional probabilities for sample 1000 / 10000
[t-SNE] Computed conditional probabilities for sample 2000 / 10000
[t-SNE] Computed conditional probabilities for sample 3000 / 10000
[t-SNE] Computed conditional probabilities for sample 4000 / 10000
[t-SNE] Computed conditional probabilities for sample 5000 / 10000
[t-SNE] Computed conditional probabilities for sample 6000 / 10000
[t-SNE] Computed conditional probabilities for sample 7000 / 10000
[t-SNE] Computed conditional probabilities for sample 8000 / 10000
[t-SNE] Computed conditional probabilities for sample 9000 / 10000
[t-SNE] Computed conditional probabilities for sample 10000 / 10000
[t-SNE] Mean sigma: 0.085168
[t-SNE] KL divergence after 250 iterations with early exaggeration: 84.923927
[t-SNE] KL divergence after 300 iterations: 3.129362
t-SNE done! Time elapsed:

In [275]:
auto_subset = np.concatenate((auto_subset, tsne_results), axis = 1)

In [281]:
plt.figure(figsize=(10, 8))

ax = sns.scatterplot(
    x = auto_subset[:, -2], 
    y = auto_subset[:, -1],
    hue = auto_subset_clusters,
    palette = sns.color_palette("hls", 20),
    legend = "full",
    alpha = 0.7
)

ax.set(title = 't-SNE of K-Means clusters of 10,000 songs (MinMax-scaled autoencoder data)', 
       xlabel = 't-SNE 1', ylabel = 't-SNE 2')
plt.savefig('tsne.png')
plt.savefig('tsne.pdf')

# TODO: The below has not been updated!
## Use K-Means to generate playlist from cold start

Given a set of start songs for the cold start, there are some potential ways of populating a playlist. For each of those songs, predict the cluster to which it belongs, then either:
- Pull songs from the mode cluster (i.e. the most common), if tied pick a random cluster and get all new songs from there.
- **Our implementation**: Pull randomly from one of the predicted clusters each time, with probability weighted equal to the number of times each cluster was predicted.
- Instead of predicting the single cluster of each start song, get its distance to all clusters, sum them up, average them, and use that to determine the closet cluster(s) of the songs as a group, then pull from either the closest one or closest $n$ with some weighting scheme (e.g. by distance).

Should handle running out of songs to sample from, i.e. if we exhaust the cluster when pulling new songs to populate the playlist, but with a reasonably low number of clusters there are so many it seems very unlikely any playlist will be so long it exhausts any cluster.

Algorithm to use a KMeans clustering model to generate a playlist. Given a fit KMeans and a set of seed songs with which to start the playlist, the algorithm predicts the cluster of each seed song using the model. It then generates a sequence of the cluster from which it will pull each new song in the playlist, weighting the probability of each cluster by the number of seed songs that belongs to that cluster. Iterating through that sequence, it pulls a random song from the respective cluster, removing it from the next iteration to avoid duplicate songs in the new playlist (however unlikely, with the dataset size). Finally, it returns the new songs in the playlist in terms of their indices in the test set and their song id in the master songs table.

In [21]:
'''
Function to generate a new playlist based on a given set of starting songs
Returns the list new songs, both as their indices in the train set and as song ids in master
Note: returns are exclusive of seed songs, so need to prepend those first if to be included
'''
def generate_playlist(start_songs, playlist_length = 20):
    # Ensure proper dimensions of songs: handles list of individual songs or set of songs
    start_clusters = kmeans.predict(np.reshape(start_songs, (len(start_songs), -1)))
    print('Starting songs clusters:', start_clusters)

    # Get count of each start cluster and its respective weight of total
    start_cluster, cluster_count = np.unique(start_clusters, return_counts = True)
    start_cluster, cluster_count = list(start_cluster), list(cluster_count)
    cluster_weight = cluster_count / np.sum(cluster_count)
    print('Cluster, Counts, Weights:', start_cluster, cluster_count, cluster_weight)

    # For each cluster from which we will sample, get indices of all songs in train set
    cluster_song_ind = [np.argwhere(train_labels == c).reshape((-1,)) for c in start_cluster]
    # Flatten component arrays
    print(len(cluster_song_ind), 'clusters with # songs each:', [len(clust) for clust in cluster_song_ind])

    # Sample list of clusters with which to populate playlist
    new_song_clusters = np.random.choice(start_cluster, size = playlist_length, p = cluster_weight)
    print('\nNew song clusters:', new_song_clusters)
    
    new_song_train_inds = np.empty((playlist_length), 'int')
    new_song_ids = np.empty((playlist_length), 'int')

    # For each new song
    print('\nRandom song from each random cluster in order:\ncluster train_ind cluster_size')
    for i, ns_c in enumerate(new_song_clusters):
        # Get index of new song cluster from list of train set song indices of each cluster to use
        ns_c_ind = start_cluster.index(ns_c)

        # Get training set index of a random new song from current cluster
        ns_train_ind = np.random.choice(cluster_song_ind[ns_c_ind], size = 1)[0]
        # Remove this song from the temporary list of cluster songs to avoid repeating songs
        cluster_song_ind[ns_c_ind] = cluster_song_ind[ns_c_ind][cluster_song_ind[ns_c_ind] != ns_train_ind]

        # Add song train set index to list of songs in new playlist
        new_song_train_inds[i] = ns_train_ind

        # Get actual song id from master songs table and add to list of new songs
        new_song_ids[i] = train_songids[ns_train_ind]

        # Print cluster of new song, its index in train, and new length of cluster
        print(ns_c, ns_train_ind, len(cluster_song_ind[ns_c_ind]))

    print('\nNew song train indices:\n', new_song_train_inds)
    print('\nNew song IDs from master table:\n', new_song_ids)
    
    return new_song_train_inds, new_song_ids


Run the function to showase how the baseline model for playlist generation works, and show the example generated playlist based on a set of previously unseen seed/starter songs.

In [22]:
# Number of songs with which to seed
songs_seed = 3

# Showcase different ways of passing in songs (list of arrays/vectors, whole array)
#start_songs = [test[i:i+1, :] for i in range(songs_seed)]
start_songs = test[0:songs_seed, :]

# Run function to generate playlist
playlist_train_indices, playlist_song_ids = generate_playlist(start_songs, 20)

Starting songs clusters: [0 0 3]
Cluster, Counts, Weights: [0, 3] [2, 1] [0.66666667 0.33333333]
2 clusters with # songs each: [66979, 175581]

New song clusters: [3 0 3 3 0 0 3 3 3 3 3 3 0 0 0 0 0 3 3 3]

Random song from each random cluster in order:
cluster train_ind cluster_size
3 778403 175580
0 373778 66978
3 913673 175579
3 628419 175578
0 353901 66977
0 100063 66976
3 384855 175577
3 323133 175576
3 387588 175575
3 306768 175574
3 183521 175573
3 31459 175572
0 75990 66975
0 890223 66974
0 221048 66973
0 614152 66972
0 698068 66971
3 720405 175571
3 607844 175570
3 552151 175569

New song train indices:
 [778403 373778 913673 628419 353901 100063 384855 323133 387588 306768
 183521  31459  75990 890223 221048 614152 698068 720405 607844 552151]

New song IDs from master table:
 [781791 375439 917610 631163 355467 100504 386561 324567 389300 308130
 184357  31600  76318 894070 222045 616840 701111 723540 610510 554565]


Show generated playlist as list of songs with metadata. Our seed/starter songs appear first, then the rest in the sequence in which they were generated.  
  
*Note: the K-Means and algorithm only use numeric values, so the non-numeric data below can be misleading as it was not taken into account.*

In [23]:
# Playlist as list of songs with metadata
pd.read_pickle('../data/master200.pkl').loc[np.append(test_songids[0:3], playlist_song_ids)]

artist_name  \
song_id                                                      
554407                                      Mint Condition   
774639   Nature Sounds Relaxation: Music for Sleep, Med...   
239936                                          Brainstorm   
781791                                       One-Eyed Doll   
375439                                           El Huervo   
917610                                           Cathedral   
631163                                         We As Human   
355467                                    Jaws Underground   
100504                                            Cascandy   
386561                                          Joe Budden   
324567                                        Oscar D'León   
389300                                         Itzik Dadya   
308130                                           Chase Coy   
184357                                       No Fun At All   
31600                                         Black Knight   
76318                                 Black Crown Initiate   
894070                                         Glass Cloud   
222045                                              Avalon   
616840                                           Dick Dale   
701111                                    Thom Sonny Green   
723540                                              C-bool   
610510                                              Tipper   
554565                                    The Long Winters   

                                    artist_uri  \
song_id                                          
554407   spotify:artist:4MT97VeycuegSicrrphisJ   
774639   spotify:artist:1qzR1EqCmsTDa9AwBLTT4b   
239936   spotify:artist:6JPBUFrfF5w5whOPk2w0kp   
781791   spotify:artist:4w2wxHsHxLp64aXeZ9v1nW   
375439   spotify:artist:2gG1LfmQPkHZNptgVpYlpr   
917610   spotify:artist:5O7ziOtiFUyRMBJN1J4Tpv   
631163   spotify:artist:6FFTGpuvFDztqTz4t1Hqy9   
355467   spotify:artist:0cqwBctx3ja0npObrVSfoU   
100504   spotify:artist:2jSv6vhlLlg9Urqffm5DwO   
386561   spotify:artist:1SAugjIcuwNPKS4urSB7A6   
324567   spotify:artist:1c84wItoiAe1pEbpJMqUmQ   
389300   spotify:artist:70287pcNpILjcpoBl0soPs   
308130   spotify:artist:5Obb6ABge8V2JOmkgQm3VY   
184357   spotify:artist:6usO1o98RY3cUWxqaQxi8O   
31600    spotify:artist:6eMofDpXdFkeb18OCEBXmq   
76318    spotify:artist:2mI89QIY20j4ly27G6qA45   
894070   spotify:artist:31FeV95p1uS4qVjd8yEHJf   
222045   spotify:artist:0SFwBkGokPRtTX212zO7xo   
616840   spotify:artist:6Ycrt8OjGSSFihsb0446eg   
701111   spotify:artist:1qXYLtBA7kceAR3kQETMUk   
723540   spotify:artist:67VM1TW9hWE9hlVYcmioaH   
610510   spotify:artist:1soJ22UMyjIw6SYFtoFJwe   
554565   spotify:artist:6zdwVjGPgGGZ7W5oBcisP2   

                                                track_name  \
song_id                                                      
554407                                Definition Of A Band   
774639   Nature Sounds – Waterfall, Trickling Cascades ...   
239936                         You Put A Charge In My Life   
781791                                           Nudie Bar   
375439                                                Rust   
917610                       Hopkins (Witchfinder General)   
631163                                       We Fall Apart   
355467                                           The Rules   
100504                                         Four - Flip   
386561                                                Wake   
324567                                 El Derecho De Nacer   
389300                                      Otcha Ani Ohev   
308130                                          Boundaries   
184357                                       Suffer Inside   
31600                                           Headphones   
76318                            To The Eye That Leads You   
894070                              How To Survive Suicide   
222045                     The Dance Temple - Original Mix   
61684